In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics.classification as M
from torch.utils.data import DataLoader, TensorDataset
from torchinfo import summary
from model_architectures import FNNEegSignals

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
import time
import os
import subprocess
from IPython import display
from scipy.signal import savgol_filter
display.set_matplotlib_formats('svg')

In [ ]:
# Change this to swap between CPU/GPU
device = torch.device('cuda:0')
# device = torch.device('cpu')

#### Read the data

In [ ]:
# Folder containing the pre-splitted folds
# dir_folds = '../../../data/outputs/eeg-signals/data-prep/partitions/3-flclients-patient-dist-os'
dir_folds = '../../../data/outputs/eeg-signals/data-prep/partitions/3-flclients'
files_names = os.listdir(dir_folds)
k_folds = int(len(files_names) / 2)

folds = {}

# Assign each fold (x, y sets) an entry in the dictionary
for fold in range(1, k_folds + 1):
    x_val = pd.read_csv(f'{dir_folds}/eeg_x_dev_flc{fold}.csv')
    y_val = pd.read_csv(f'{dir_folds}/eeg_y_dev_flc{fold}.csv')
    folds[fold] = {'x_val': x_val, 'y_val': y_val}

In [ ]:
# fnn = FNNEggSignals(4, 64)
# summary(fnn, input_data=features, device=device)

#### Training/Evaluation Functions

In [ ]:
def get_vram_usage():
    try:
        cmd = ['nvidia-smi', '--query-gpu=memory.used', '--format=csv,noheader,nounits']
        result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        if result.returncode == 0:
            vram_used = int(result.stdout.strip())
            return vram_used
        else:
            print("Error:", result.stderr, flush='True')
    except Exception as e:
        print("An error occurred:", e, flush='True')
    return None

def train(model, x_train, y_train, x_val, y_val, exp_conf, exp_name):
    # Get the value to scale the weights based on class imbalance
    n_pos = torch.tensor(y_train[y_train==1].shape[0]).float()
    n_neg = torch.tensor(y_train[y_train==0].shape[0]).float()
    pos_weight = n_neg / n_pos

    # Loss Function
    loss_function = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    # loss_function = nn.BCEWithLogitsLoss()

    # Optimizer
    optimizer = getattr(torch.optim, exp_conf['optimizer'])
    optimizer = optimizer(model.parameters(), lr=exp_conf['learning_rate'])

    # Metrics Accumulators
    train_losses = []
    train_accuracies = []
    train_sensitivities = []
    train_specificities = []
    val_losses = []
    val_accuracies = []
    val_sensitivities = []
    val_specificities = []

    # Define evaluation metrics settings
    accuracy_metric = M.BinaryAccuracy(threshold=exp_conf['decision_boundary']).to(device)
    recall_metric = M.BinaryRecall(threshold=exp_conf['decision_boundary']).to(device)
    specificity_metric = M.BinarySpecificity(threshold=exp_conf['decision_boundary']).to(device)

    # For each training epoch
    for epoch_i in range(exp_conf['n_epochs']):
        ## Training
        model.train()
        
        # Update on progress
        print(f'Running {exp_name}, epoch {epoch_i} of {exp_conf["n_epochs"]-1}')
        display.clear_output(wait=True)
        
        # Forward Pass
        y_hat = model(x_train)

        # Compute the Loss
        train_loss = loss_function(y_hat, y_train)
        train_losses.append(train_loss)

        # Backpropagation
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        ## Compute epoch training metrics
        y_pred = torch.sigmoid(y_hat)
        train_accuracy = accuracy_metric(y_pred, y_train)
        train_accuracies.append(train_accuracy)
        train_sensitivity = recall_metric(y_pred, y_train)
        train_sensitivities.append(train_sensitivity)
        train_specificity = specificity_metric(y_pred, y_train)
        train_specificities.append(train_specificity)

        ## Compute epoch evaluation metrics
        val_results = evaluate(model, x_val, y_val, exp_conf)
        val_losses.append(val_results['loss'])
        val_accuracies.append(val_results['accuracy'])
        val_sensitivities.append(val_results['sensitivity'])
        val_specificities.append(val_results['specificity'])

        vram_used = get_vram_usage()
        dict_memory = {
            'epoch': [epoch_i]
            , 'VRAM': [vram_used]
        }

        # print(dict_memory, flush=True
        df_memory = pd.DataFrame.from_dict(dict_memory)
        filename = f'{exp_conf["logs_path"]}/fnn_memory_centralized.csv'
        df_memory.to_csv(filename, index=False, mode='a', header=not os.path.exists(filename))

    # Collect all objects in CPU
    result = {
        # Training
        'train_losses': torch.as_tensor(train_losses).cpu()
        , 'train_accuracies': torch.as_tensor(train_accuracies).cpu()
        , 'train_sensitivities': torch.as_tensor(train_sensitivities).cpu()
        , 'train_specificities': torch.as_tensor(train_specificities).cpu()
        # Validation
        , 'val_losses': torch.as_tensor(val_losses).cpu()
        , 'val_accuracies': torch.as_tensor(val_accuracies).cpu()
        , 'val_sensitivities': torch.as_tensor(val_sensitivities).cpu()
        , 'val_specificities': torch.as_tensor(val_specificities).cpu()
    }

    return result

def evaluate(model, x_val, y_val, exp_conf):
    model.eval()
    # Get the value to scale the weights based on class imbalance
    n_pos = torch.tensor(y_val[y_val==1].shape[0]).float()
    n_neg = torch.tensor(y_val[y_val==0].shape[0]).float()
    pos_weight = n_neg / n_pos

    # Loss Function
    loss_function = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    # loss_function = nn.BCEWithLogitsLoss()

    with torch.no_grad():
        y_hat = model(x_val)
        loss = loss_function(y_hat, y_val)
        y_pred = torch.sigmoid(y_hat)

        # Define evaluation metrics settings
        accuracy_metric = M.BinaryAccuracy(threshold=exp_conf['decision_boundary']).to(device)
        recall_metric = M.BinaryRecall(threshold=exp_conf['decision_boundary']).to(device)
        specificity_metric = M.BinarySpecificity(threshold=exp_conf['decision_boundary']).to(device)

        # Compute metrics
        accuracy = accuracy_metric(y_pred, y_val)
        sensitivity = recall_metric(y_pred, y_val)
        specificity = specificity_metric(y_pred, y_val)

        results = {'loss': loss, 'accuracy': accuracy, 'sensitivity': sensitivity, 'specificity': specificity}

    return results

def compile_results(epoch_results, exp_conf, elapsed_time, filepath):
    ## Training Results (Wide Format)
    results_wide_df = pd.DataFrame.from_dict(epoch_results, orient='columns')
    results_wide_df['epoch'] = [epoch for epoch in range(exp_conf['n_epochs'])]
    results_wide_df['exp_name'] = exp_conf['experiment_name']

    ## Training Results (Long Format)
    results_long_df = pd.melt(frame=results_wide_df, id_vars=['epoch', 'exp_name'], value_vars=epoch_results.keys(), var_name='metric', value_name='value')
    results_long_df = results_long_df.sort_values('epoch')

    ## Training Summary 
    epoch_best_model = epoch_results['val_losses_mean'].argmin()
    
    summary_exp = {
        'exp_name': [exp_conf['experiment_name']]
        , 'exp_type': [exp_conf['experiment_type']]
        , 'exp_resource': [exp_conf['resource']]
        , 'exp_device': [exp_conf['device']]
        , 'exp_k_folds': [exp_conf['k-folds']]
        , 'exp_configuration': [json.dumps(exp_conf)]
        , 'elapsed_time': [elapsed_time]
        , 'epoch_best_model': epoch_best_model
    }

    # Get the aggregated metrics for the epoch with the best model performance
    for metric_name in epoch_results.keys():
        if('mean' in metric_name):
            metric_value = epoch_results[metric_name][epoch_best_model]
            summary_exp[metric_name] = [metric_value]

    exp_summary_df = pd.DataFrame.from_dict(summary_exp, orient='columns')

    # Persist experiments to disk (If path is provided)
    if(filepath!=None):
        filename_train_wide = f'{filepath}/results_train_wide.csv'
        results_wide_df.to_csv(filename_train_wide, index=False, mode='a', header=not os.path.exists(filename_train_wide))
        filename_train_long = f'{filepath}/results_train_long.csv'
        results_long_df.to_csv(filename_train_long, index=False, mode='a', header=not os.path.exists(filename_train_long))
        filename_summary = f'{filepath}/exp_summary_train.csv'
        exp_summary_df.to_csv(filename_summary, index=False, mode='a', header=not os.path.exists(filename_summary))
    
    return {
        'results': {'wide': results_wide_df, 'long': results_long_df}
        , 'summary_exp': exp_summary_df
    }

#### Model Configuration

In [ ]:
# Model Configuration
exp_conf = {
    # Experiment Metadata
    'experiment_name': '2L_64U_Adam_1e-2_E3000_BSS_WeightedLoss'
    , 'experiment_type': 'Centralized'
    , 'resource': 'Cloud'
    , 'device': device.type
    , 'k-folds': k_folds
    # Architecture Parameters
    , 'n_epochs': 3000
    , 'n_layers': 2
    , 'n_units': 64
    , 'learning_rate': 1e-2
    , 'decision_boundary': 0.5
    , 'perc_dropout': 0.0
    , 'optimizer': 'Adam'
    , 'loss_function': 'BCEWithLogitsLoss'
} 

#### Fit the Model

In [ ]:
# Accumulators
fold_results = {
    'train_losses': [], 'train_accuracies': [], 'train_sensitivities': [], 'train_specificities': []
    , 'val_losses': [], 'val_accuracies': [], 'val_sensitivities': [], 'val_specificities': []
}

elapsed_times = []

# Per K-Fold
for fold in folds.keys():
    # Select the current fold for validation 
    x_val = folds[fold]['x_val']
    y_val = folds[fold]['y_val']

    # Union the rest of the folds for training
    train_folds = {key: value for key, value in folds.items() if key != fold}
    x_train = pd.DataFrame()
    y_train = pd.DataFrame()

    for train_fold in train_folds:
        x_train = pd.concat([x_train, train_folds[train_fold]['x_val']])
        y_train = pd.concat([y_train, train_folds[train_fold]['y_val']])

    # Z-Score Normalization
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_val = scaler.fit_transform(x_val)

    # Load the current fold sets into PyTorch Tensors
    x_train = torch.tensor(x_train, device=device).float()
    y_train = torch.tensor(y_train.values, device=device).float()
    x_val = torch.tensor(x_val, device=device).float()
    y_val = torch.tensor(y_val.values, device=device).float()

    # Create a fresh instance of the model
    fnn = FNNEegSignals(exp_conf['n_layers'], exp_conf['n_units'], exp_conf['perc_dropout'])
    fnn.to(device)

    fold_start_time = time.time()
    # Train this fold
    result = train(fnn, x_train, y_train, x_val, y_val, exp_conf, f'{exp_conf["experiment_name"]} Fold {fold}')
    fold_end_time = time.time()
    fold_elapsed_time = fold_end_time - fold_start_time
    elapsed_times.append(fold_elapsed_time)

    # Accumulate metrics
    for metric in result.keys():    
        fold_results[metric].append(result[metric])

    # Free GPU memory after each run
    del fnn, x_train, y_train, x_val, y_val
    torch.cuda.empty_cache() 
    gc.collect()

## Aggregate Fold Measures
agg_fold_results = {}

for metric_name in fold_results.keys():
    metric_matrix = np.array([]).reshape(0, exp_conf['n_epochs'])
    
    for fold, metric in enumerate(fold_results[metric_name]):
        metric = metric.reshape(1, exp_conf['n_epochs'])
        metric_matrix = np.vstack([metric_matrix, metric])
    
    metric_mean = metric_matrix.mean(axis=0)
    metric_sd = metric_matrix.std(axis=0)
    agg_fold_results[f'{metric_name}_mean'] = metric_mean
    agg_fold_results[f'{metric_name}_std'] = metric_sd

epoch_results = {key: agg_fold_results[key] for key in agg_fold_results.keys() if 'mean' in key}  

In [ ]:
elapsed_time = np.array(elapsed_times).sum()
filepath = '../../../data/logs/experiments/eegsignals_ffn'
# filepath = None # Disable Persistence

# Create path if it doesn't exist
if((filepath!=None) & (~os.path.exists(filepath))):
    os.makedirs(filepath)
    
comp_train_results = compile_results(agg_fold_results, exp_conf, elapsed_time, filepath)

In [ ]:
figs, axs = plt.subplots(1, 3, figsize=(14, 4))
axs = axs.flatten()

for fold in range(0, k_folds):
    axs[fold].set_title(f'Fold {fold+1} Loss')
    axs[fold].set_xlabel('Epoch')
    axs[fold].set_ylabel('Loss')
    axs[fold].plot(fold_results['train_losses'][fold], label=f'Train', color='C0')
    axs[fold].plot(fold_results['val_losses'][fold], label=f'Val', color='C1')
    axs[fold].legend()

plt.tight_layout()
plt.show()

In [ ]:
comp_train_results['summary_exp']